## There are some instructions you need to follow:
<li> You only need to write your code in the comment area "Your Code Here".</li>
<li>Do not upload your own file. Please make the necessary changes in the Jupyter notebook file already present in the server.</li>
<li>Please note, there are several cells in the Assignment Jupyter notebook that are empty and read only. Do not attempt to remove them or   edit them. They are used in grading your notebook. Doing so might lead to 0 points.</li>

In [1]:
import nltk
import os
import _sqlite3
from nltk.corpus import PlaintextCorpusReader
from nltk import sent_tokenize,word_tokenize 
from gensim import corpora, models, similarities
from gensim.models.ldamodel import LdaModel
from gensim.parsing.preprocessing import STOPWORDS
from gensim.similarities.docsim import Similarity

# Question 1

In [2]:
"""
Question 1

Write a function that takes the file name of the Wikipedia page containing all Greek ancient
philosophers (saved as "Index.html" in your workspace) and returns a list tuples containing 
the name of the philosopher and the path to its individual article file.

Example of use: get_philosophers("Index.html")

The output should be a list of tuples:

[('Acrion', 'Philosophers/Acrion.html'),
 ('Adrastus of Aphrodisias', 'Philosophers/Adrastus of Aphrodisias.html'),
 ('Aedesia', 'Philosophers/Aedesia.html'),
 ('Aedesius', 'Philosophers/Aedesius.html'),
 ('Aeneas of Gaza', 'Philosophers/Aeneas of Gaza.html'),
 ('Aenesidemus', 'Philosophers/Aenesidemus.html'),
 ...]
 
  
NOTE: For processing speed purposes, the table in "Index.html" has been shortened compared
to the one online on wikipedia.org. Do not worry if you do not find some philosophers in 
your results, this is made on purpose. 

"""

def get_philosophers(filename):
    
    import codecs
    from bs4 import BeautifulSoup
    f = codecs.open(filename, 'r', 'utf-8')
    soup = BeautifulSoup(f.read(),'lxml')
    
    philosopher_list = []
    
    tableRows = soup.find_all('table',class_='wikitable sortable')[0]
    data_rows = tableRows.find_all('tr')[1:]
    philosopher_name = [] #list containing each philosopher's name
    # grabs philosopher name from title an appends to list
    for data in data_rows:
        philosopher_name.append(data.find('a').get('title'))

    # adding path and filename at the end of philosopher's name
    for name in philosopher_name:
        x = name
        y = name,"Philosophers/"+ x + '.html'
        philosopher_list.append(y)

    # returns philosopher list as tuple
#     philosopher_tuple_list = tuple(philosopher_list)
    return philosopher_list

# Once done, try this:
filenames = get_philosophers("Index.html")
# filenames

# Question 1: Non-Function

In [3]:
import codecs
from bs4 import BeautifulSoup
f = codecs.open("Index.html", 'r', 'utf-8')
soup = BeautifulSoup(f.read(),'lxml')

# reference: http://savvastjortjoglou.com/nba-draft-part01-scraping.html
# tuples of philosopher name w/ path 
philosopher_list = []

# creates list of philosopher name
tableRows = soup.find_all('table',class_='wikitable sortable')[0]
data_rows = tableRows.find_all('tr')[1:]
philosopher_name = [] #list containing each philosopher's name

# grabs philosopher name from title and appends to list
for data in data_rows:
    philosopher_name.append(data.find('a').get('title'))
#     print(type(data))


# # loops through a list indices 
# for i in range(len(data_rows)):
#     # for each table data element from each table row
#     td = data_rows[i].find_all('td')[0].get_text().strip()
# #     philosopher_name.append(td)


# adding path and filename at the end of philosopher's name
for name in philosopher_name:
    x = name
    y = name,"Philosophers/"+ x + '.html'
    philosopher_list.append(y)

philosopher_list[84]


('Xenophilus', 'Philosophers/Xenophilus.html')

# Question 2

In [4]:
"""
Question 2


Write a function that scrapes the text on a philosophers’s page and returns it as a text 
string. The input is the name of the file that contains the philosopher's page.

Example of use: get_text('Philosophers/Acrion.html')
should output the text of the page.
'Acrion was a Locrian and a Pythagorean philosopher...'
"""

def get_text(file):
    import codecs
    from bs4 import BeautifulSoup
    f = codecs.open(file, 'r', 'utf-8')
    page_soup = BeautifulSoup(f.read(),'lxml')

    all_text = ""

    for tag in page_soup.find_all('p'):
        all_text += tag.get_text()
    return all_text
# Once done, try this:
# get_text("Philosophers/Acrion.html")

# Question 2: Non-Function (all p tags)

In [58]:
# # looks at page index.html (all philosophers)
# filename = "Philosophers/Acrion.html"

# # import codecs
# # from bs4 import BeautifulSoup
# # f = codecs.open(filename, 'r', 'utf-8')
# # page_soup = BeautifulSoup(f.read(),'lxml')


# all_text = ""

# # takes the 2nd index from list philosopher_list
# for philosopher_filepath in philosopher_list:
#     philosopher_filepath = philosopher_filepath[1]
    
#     filepath = codecs.open(philosopher_filepath, 'r', 'utf-8')
#     page_soup = BeautifulSoup(filepath.read(),'lxml')

# # find the first p-tag
#     for tag in page_soup.find_all('p'):
#         all_text += tag.get_text()
# # all_text



# Question 2: Non-Function (first p tag)

In [74]:
all_text = ""

# takes the 2nd index from list philosopher_list
for philosopher_filepath in philosopher_list:
    philosopher_filepath = philosopher_filepath[1]
    
    filepath = codecs.open(philosopher_filepath, 'r', 'utf-8')
    page_soup = BeautifulSoup(filepath.read(),'lxml')
    
    # find the first p-tag
    for tag in page_soup.find_all('p'):
        print(tag.get_text())
#         all_text += tag.get_text()
# all_text

Acrion was a Locrian and a Pythagorean philosopher.[1]  He is mentioned by Valerius Maximus[2] under the name of Arion. According to William Smith, Arion is a false reading, instead of Acrion.[3]

Adrastus of Aphrodisias (Greek: Ἄδραστος ὁ Ἀφροδισιεύς; fl. 2nd century) was a Peripatetic philosopher who lived in the 2nd century AD.  He was the author of a treatise on the arrangement of Aristotle's writings and his system of philosophy, quoted by Simplicius,[1] and by Achilles Tatius.  Some commentaries of his on the Timaeus of Plato are also quoted by Porphyry,[2] and a treatise on the Categories of Aristotle by Galen. None of these have survived.[3] He was a competent mathematician, whose writings on harmonics are frequently cited by Theon of Smyrna in the surviving sections of his On Mathematics Useful for the Understanding of Plato.[4]  In the 17th century, a work by Adrastus on harmonics, Περὶ Ἁρμονικῶν ("On Harmonics"), was said by Gerhard Johann Vossius to have been preserved, in 

Alcinous (Greek: Ἀλκίνοος Alkinoos) was a Middle Platonist philosopher.  He probably lived in the 2nd century AD,[1] although nothing is known about his life.[2] He is the author of The Handbook of Platonism, an epitome of Middle Platonism intended as a manual for teachers. He has, at times, been identified by some scholars with the 2nd century Middle Platonist Albinus.

Alcinous is the author of work called The Handbook of Platonism (Ἐπιτομὴ τῶν Πλάτωνος δογμάτων, also Διδασκαλικὸς τῶν Πλάτωνος δογμάτων; Latin: De doctrina Platonis), one of the few surviving works from the Middle Platonist period (c. 90 BC – 250 AD).  The book contains 36 chapters which cover topics ranging from logic to physics to ethics.  It is thought to have been intended as a manual, not for students of Platonism, but for its teachers.  The treatise is written in the esoteric manner typical of the Corpus Aristotelicum, and it often appropriates popular concepts from other philosophical schools — in particular the

Ammonius Hermiae (/əˈmoʊniəs/; Greek: Ἀμμώνιος ὁ Ἑρμείου Ammōnios ho Hermeiou; c. 440 – c. 520) was a Greek philosopher, and the son of the Neoplatonist philosophers Hermias and Aedesia. He was a pupil of Proclus in Athens, and taught at Alexandria for most of his life, writing commentaries on Plato, Aristotle, and other philosophers.

Ammonius' father, Hermias, died when he was a child, and his mother, Aedesia, raised him and his brother, Heliodorus, in Alexandria. When they reached adulthood, Aedesia accompanied her sons to Athens where they studied under Proclus. Eventually, they returned to Alexandria, where Ammonius, as head of the Neoplatonist school in Alexandria, lectured on Plato and Aristotle for the rest of his life. According to Damascius, during the persecution of the pagans at Alexandria in the late 480s, Ammonius made concessions to the Christian authorities so that he could continue his lectures.[1] Damascius, who scolds Ammonius for the agreement that he made, does not

Anaximander (/æˌnæksɪˈmændər/; Greek: Ἀναξίμανδρος Anaximandros; c. 610 – c. 546 BC), was a pre-Socratic Greek philosopher who lived in Miletus,[4] a city of Ionia (in modern-day Turkey). He belonged to the Milesian school and learned the teachings of his master Thales. He succeeded Thales and became the second master of that school where he counted Anaximenes and, arguably, Pythagoras amongst his pupils.[5]

Little of his life and work is known today. According to available historical documents, he is the first philosopher known to have written down his studies,[6] although only one fragment of his work remains. Fragmentary testimonies found in documents after his death provide a portrait of the man.

Anaximander was an early proponent of science and tried to observe and explain different aspects of the universe, with a particular interest in its origins, claiming that nature is ruled by laws, just like human societies, and anything that disturbs the balance of nature does not last lo

Anniceris (Greek: Ἀννίκερις; fl. 300 BC) was a Cyrenaic philosopher. He argued that pleasure is achieved through individual acts of gratification which are sought for the pleasure that they produce, but he also laid great emphasis on the love of family, country, friendship and gratitude, which provide pleasure even when they demand sacrifice. 

Anniceris was a disciple of Parabates, and a fellow student of Hegesias. The Suda says he lived at the time of Alexander the Great (ruled 336–323 BC).[1] The story that Anniceris ransomed Plato from Dionysius, tyrant of Syracuse for twenty minas,[2] must refer to an earlier Anniceris, possibly the celebrated charioteer mentioned by Aelian.[3]

Anniceris denied that pleasure was merely the absence of pain, for if so death would be a pleasure; and furthermore he denied that pleasure is the general goal of human life. To each separate action, there is a particular end, namely the pleasure which actually results from it.[4] He differed from Aristipp

...

Aristo of Chios (Greek: Ἀρίστων ὁ Χῖος Ariston ho Chios; fl. c. 260 BC) was a Stoic philosopher and colleague of Zeno of Citium. He outlined a system of Stoic philosophy that was, in many ways, closer to earlier Cynic philosophy. He rejected the logical and physical sides of philosophy endorsed by Zeno and emphasized ethics. Although agreeing with Zeno that Virtue was the supreme good, he rejected the idea that morally indifferent things such as health and wealth could be ranked according to whether they are naturally preferred. An important philosopher in his day, his views were eventually marginalized by Zeno's successors.

Aristo, son of Miltiades, was born on the island of Chios sometime around 300 BC.[1] He came to Athens where he attended the lectures of Zeno of Citium, and also, for a time, the lectures of Polemo,[2] (the head of the Academy from 314 to 269). Although he was a member of Zeno's circle he soon departed from Zeno's teachings, largely rejecting the two non-ethical p





Aristotle (/ˈærɪstɒtəl/;[3] Greek: Ἀριστοτέλης Aristotélēs, pronounced [aristotélɛːs]; 384–322 BC)[A] was a Greek philosopher during the Classical period in Ancient Greece, the founder of the Lyceum and the Peripatetic school of philosophy and Aristotelian tradition. Along with his teacher Plato, he has been called the "Father of Western Philosophy". His writings cover many subjects – including physics, biology, zoology, metaphysics, logic, ethics, aesthetics, poetry, theatre, music, rhetoric, psychology, linguistics, economics, politics and government. Aristotle provided a complex synthesis of the various philosophies existing prior to him, and it was above all from his teachings that the West inherited its intellectual lexicon, as well as problems and methods of inquiry. As a result, his philosophy has exerted a unique influence on almost every form of knowledge in the West and it continues to be a subject of contemporary philosophical discussion.

Little is known about his life.

Athenodoros Cordylion (or Athenodorus, Ancient Greek: Αθηνόδωρος Κορδυλίων; fl. early-mid 1st century BC) was a Stoic philosopher, born in Tarsus. He was the keeper of the library at Pergamon, where he was known to cut out passages from books on Stoic philosophy if he disagreed with them:[1]

The passages disapproved by the school were expunged from his [Zeno's] works by Athenodorus the Stoic, who was in charge of the Pergamene library; and that afterwards, when Athenodorus was detected and compromised, they were replaced.
In his old age, Athenodorus relocated to Rome, where he lived with Cato the Younger until his death.[2][3]



Athenodorus of Soli (Greek: Ἀθηνόδωρος ὁ Σολεύς) was a Stoic philosopher, and disciple of Zeno of Citium, who lived in the 3rd century BC.

He was the son of Athenodorus, and was born in the town of Soli, Cilicia, and was the compatriot of another disciple of Zeno, Chrysippus. Athenodorus was the brother of the poet Aratus of Soli,[1] the author of the long d

Xenophanes of Colophon (/zəˈnɒfəniːz/;[1][2] Ancient Greek: Ξενοφάνης ὁ Κολοφώνιος [ksenopʰánɛːs ho kolopʰɔ̌ːnios]; c. 570 – c. 475 BC)[3] was a Greek philosopher, theologian, poet, and social and religious critic. Xenophanes lived a life of travel, having left Ionia at the age of 25 and continuing to travel throughout the Greek world for another 67 years.[4] Some scholars say he lived in exile in Sicily.[5] Knowledge of his views comes from fragments of his poetry, surviving as quotations by later Greek writers. To judge from these, his elegiac and iambic[6] poetry criticized and satirized a wide range of ideas, including Homer and Hesiod, the belief in the pantheon of anthropomorphic gods and the Greeks' veneration of athleticism. He is the earliest Greek poet who claims explicitly to be writing for future generations, creating "fame that will reach all of Greece, and never die while the Greek kind of songs survives."[7]

Xenophanes was a native of Colophon, a city in Ionia (now west

Zeno of Elea (/ˈziːnoʊ ... ˈɛliə/; Greek: Ζήνων ὁ Ἐλεάτης; c. 495 – c. 430 BC[1]) was a pre-Socratic Greek philosopher of Magna Graecia and a member of the Eleatic School founded by Parmenides.  Aristotle called him the inventor of the dialectic.[2] He is best known for his paradoxes, which Bertrand Russell has described as "immeasurably subtle and profound".[3]

Little is known for certain about Zeno's life. Although written nearly a century after Zeno's death, the primary source of biographical information about Zeno is Plato's Parmenides[4] and he is also mentioned in Aristotle's Physics.[5] In the dialogue of Parmenides, Plato describes a visit to Athens by Zeno and Parmenides, at a time when Parmenides is "about 65", Zeno is "nearly 40", and Socrates is "a very young man".[6] Assuming an age for Socrates of around 20 and taking the date of Socrates' birth as 469 BC gives an approximate date of birth for Zeno of 490 BC. Plato says that Zeno was "tall and fair to look upon" and was 

# Question 3 (functional)

In [6]:
# """
# Question 3

# Use the files under "Philosophers" folder to construct an LSI model.
# Then, use the LSI model to find the most similar philosopher for each of the philosophers
# found in Question 1, based on the content of their Wikipedia articles. You should not go
# online to scrape the data; everything you need is in your Jupyter notebook working directory.

# The function should have as input the list of tuples created in Question 1.

# The output format should be a list of tuples too. Each tuple should contain a philosopher's name
# and its most similar other philosopher. Please note both names can't be the same.
# """

"""
The output should look like that:

[('Acrion', 'Arignote'),
 ('Adrastus of Aphrodisias', 'Lycophron (Sophist)'),
 ('Aedesia', 'Heliodorus of Alexandria'),
 ('Aedesius', 'Chrysanthius'),
 ('Aeneas of Gaza', 'Archytas'),
 ...]
"""

# def run1(filenames):
# ###
# ### YOUR CODE HERE
# ###

# # Once done, try this:
# run(filenames)

"\nThe output should look like that:\n\n[('Acrion', 'Arignote'),\n ('Adrastus of Aphrodisias', 'Lycophron (Sophist)'),\n ('Aedesia', 'Heliodorus of Alexandria'),\n ('Aedesius', 'Chrysanthius'),\n ('Aeneas of Gaza', 'Archytas'),\n ...]\n"

# Question 3 (non-functional)

In [7]:
import nltk
import os
import _sqlite3
from nltk.corpus import PlaintextCorpusReader
from nltk import sent_tokenize,word_tokenize 
from gensim import corpora, models, similarities
from gensim.models.ldamodel import LdaModel
from gensim.parsing.preprocessing import STOPWORDS
from gensim.similarities.docsim import Similarity



In [26]:
# list of all philosophers' file path
philosopher_files = list()
for filename in filenames:
    philosopher_files.append(filename[1])

# grab each philosopher's <p> and append to list
# user defined function: get_text()
# philosopher_text_list = list()
# for philosopher_path in philosopher_files:
#     philosopher_text_list.append(get_text(philosopher_path))

# list of philosophers' text 
documents = [get_text(philosopher_path) for philosopher_path in philosopher_files]

In [32]:
# text preprocessing
# philosopher_text_list = contains multiple indexes of philosophers' description
strip_text_list = list()

for philosopher_text in documents:
    striptext = philosopher_text.replace('\n\n', ' ')
    striptext = striptext.replace('\n', ' ')

##     tokenize all texts: only used when creating LDA (sent_tokenize)
#     sentences = sent_tokenize(striptext)

    strip_text_list.append(striptext)

# philosopher acrion is most similar to
# strip_text_list


In [44]:
# lowercase all characters, split the words and remove all special characters
# removed: "word not in STOPWORDS and"
texts = [[word for word in document.lower().split()
         if word not in STOPWORDS and word.isalnum()]
         for document in strip_text_list]

# create word dictionary for each word in a text 
# tokenizing
dictionary = corpora.Dictionary(texts)

# create corpus: assigns ints to words
# https://radimrehurek.com/gensim/corpora/dictionary.html#gensim.corpora.dictionary.Dictionary.doc2bow
# returns (token_id,token_count)
corpus = [dictionary.doc2bow(text) for text in texts]


In [46]:
# train LSI model
# id2word takes a dictionary of words
lsi = models.LsiModel(corpus, id2word=dictionary,num_topics=2)

# ex) index 1 [Acrion]
# text_acrion_bow = [[word for word in document.lower()
#          if word not in STOPWORDS and word.isalnum()]
#          for document in strip_text_list[0]]

text_acrion_bow = []
acrion_words_list = strip_text_list[0].split()
for word in acrion_words_list:
    if word.isalnum() == True:
        text_acrion_bow.append(word)

# vectorize new bow (bag of words)
acrion_vec_bow = dictionary.doc2bow(text_acrion_bow)
vec_lsi = lsi[acrion_vec_bow]
index = similarities.MatrixSimilarity(lsi[corpus])
sims = index[vec_lsi]
sims = sorted(enumerate(sims),key=lambda item: -item[1])




In [47]:
# acrion is most similar to index 75 = Athenodorus of Soli
sims

[(75, 0.9999115),
 (8, 0.99764264),
 (55, 0.99389684),
 (7, 0.9929013),
 (53, 0.9917134),
 (84, 0.9904734),
 (40, 0.98420525),
 (66, 0.9833431),
 (41, 0.98281384),
 (39, 0.977679),
 (90, 0.9769584),
 (2, 0.9760174),
 (67, 0.9750929),
 (57, 0.9748107),
 (59, 0.97377586),
 (12, 0.9724612),
 (44, 0.97212505),
 (48, 0.9718428),
 (69, 0.9711671),
 (26, 0.97099),
 (24, 0.97088516),
 (43, 0.97019947),
 (9, 0.9684013),
 (42, 0.9676485),
 (76, 0.9669211),
 (86, 0.96555626),
 (3, 0.96446276),
 (88, 0.9590136),
 (21, 0.95733833),
 (45, 0.95649356),
 (38, 0.9563924),
 (47, 0.95436805),
 (11, 0.9535912),
 (65, 0.9527466),
 (37, 0.95259184),
 (36, 0.9521341),
 (74, 0.9508228),
 (83, 0.9477255),
 (50, 0.9465109),
 (78, 0.9464924),
 (85, 0.941419),
 (91, 0.9327191),
 (56, 0.9320204),
 (27, 0.9270043),
 (58, 0.924785),
 (18, 0.9204238),
 (79, 0.91870624),
 (72, 0.918166),
 (14, 0.91288424),
 (51, 0.9128355),
 (22, 0.91213596),
 (34, 0.9112351),
 (73, 0.91067874),
 (6, 0.9105065),
 (29, 0.909552),
 (5, 

In [36]:
# output name of philosopher and name of most similar philosopher

In [13]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [14]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [15]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###
